### 1. 패키지 불러오기

#### 설치가 되지 않은 패키지는 anaconda에서 pip install로 설치가능합니다.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from collections import Counter
from wordcloud import WordCloud
from konlpy.tag import Okt
import networkx as nx
from gensim.models import doc2vec
import os
import pyLDAvis.gensim 
import gensim.corpora as corpora
import gensim
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans 
from sklearn.preprocessing import MinMaxScaler

In [2]:
import matplotlib.font_manager as fm
from matplotlib import rc
font_name = fm.FontProperties(fname="c:/Windows/Fonts/H2GTRE.ttf").get_name()
rc('font', family=font_name)

### 2. 데이터 불러오기

In [3]:
def year_extract(date):
    return int(date[:4])

In [4]:
cookit = pd.read_excel('cookit.xlsx')
cookit['year'] = cookit['date'].map(year_extract)
cookit = cookit.iloc[:, [0,1,6,7, -1]]
cookit.columns = ['company', 'product_name', 'score', 'review', 'year']
cookit = cookit.loc[cookit['year'] < 2022, :]
cookit = cookit.loc[cookit['year'] > 2018, :].reset_index(drop = True)
cookit.head()

C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,company,product_name,score,review,year
0,cookit,[사전예약] 스파이시 보일링랍스터&쉬림프,3,기대가 너무 커서 그랬는지 제 입맛엔 그냥 ...쿡킷 메뉴 대부분 마음에 드는데 이...,2021
1,cookit,[사전예약] 리코타치즈 콜드파스타,5,아 이거 진짜 맛있네요!남편과 저는 물론이고 5살 딸도 너무너무 맛있게 잘 먹었어요...,2021
2,cookit,수삼양지수육전골,5,추천해요~국물도 따뜻하고 버섯이랑 고기 넉넉해서 한끼 잘 먹었습니다!! 맛있게 잘 ...,2021
3,cookit,평양식 어복쟁반,5,맛있게 먹고 재주문해요아이부터 어른까지 온가족 맛있게 먹었어요레시피 보고 삶은달걀인...,2021
4,cookit,감바스알아히요,5,감바스 재료를 따로 사지않고 한번에 만들어 먹을 수 있어서 좋아요 !살짝 매콤하게 ...,2021


In [5]:
fresheasy = pd.read_excel('fresheasy.xlsx')
fresheasy['year'] = fresheasy['date'].map(year_extract)
fresheasy = fresheasy.iloc[:, [0,1,5,6, -1]]
fresheasy.columns = ['company', 'product_name', 'score', 'review', 'year']
fresheasy = fresheasy.loc[fresheasy['year'] < 2022, :]
fresheasy = fresheasy.loc[fresheasy['year'] > 2018, :].reset_index(drop = True)
fresheasy.head()

C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,company,product_name,score,review,year
0,fresheasy,블랙라벨 스테이크 세트,5,항상 신선식품 감사합니다 블랙라벨 스테이크는 후회없이 매달 시켜먹는중이에요 ^^,2021
1,fresheasy,블랙라벨 스테이크 세트,4,가격대비 나쁘지 않았어요 괜찮았어요 두번 구매는 잘 모르겠네요 호불호가 갈릴거같아요,2021
2,fresheasy,블랙라벨 스테이크 세트,5,좋아요. 아직 먹어보진 못햇습니다. 큐커로 해벌생각하니 벌써부터 기대가 됩니다.,2021
3,fresheasy,블랙라벨 스테이크 세트,5,가격대비 고기퀄리티가 훌륭합니다!! 스테이크 치고 가격이 저렴해서 큰 기대는 하지 ...,2021
4,fresheasy,블랙라벨 스테이크 세트,5,매우만족 역시 스테디셀러는 이유가 있네요 큰 기대안했는데 너무 맛있게 먹었어요 고기...,2021


In [6]:
mychef = pd.read_excel('mychef.xlsx')
mychef['year'] = mychef['date'].map(year_extract)
mychef = mychef.iloc[:, [0,1,4,7,-1]]
mychef.columns = ['company', 'product_name', 'score', 'review', 'year']
mychef = mychef.loc[mychef['year'] < 2022, :]
mychef = mychef.loc[mychef['year'] > 2018, :].reset_index(drop = True)
mychef.head()

C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,company,product_name,score,review,year
0,mychef,[마이셰프X큐커] 스페니시 감바스 알아히요(2인),4,감바스 굿! 전체적인 맛과 향은 넘 괜찮아요! 단.먹다보면 살짝 느끼한듯해서 와인하...,2021
1,mychef,[마이셰프X큐커] 스페니시 감바스 알아히요(2인),5,맛있어요. 추천합니다. 달콤 짭쪼름하니 맛있네요. 가격은 좀 비싼편이지만 한끼 식사...,2021
2,mychef,[마이셰프X큐커] 스페니시 감바스 알아히요(2인),4,맛나요 재구매입니다 조리가 편한데 이것도 큐커 바코드 인식이 안되네요ㅠㅠ 그럼 몇도...,2021
3,mychef,[마이셰프X큐커] 스페니시 감바스 알아히요(2인),5,너무 맛있게 먹었어요! 큐커 들이고나서 마이쉐프에서 주문해서 먹었는데사용법도 너무 ...,2021
4,mychef,[마이셰프X큐커] 스페니시 감바스 알아히요(2인),4,간편해요~ 큐커 밀키트 중 젤 간편하네요 첨부파일 20210925_191729.jpg,2021


In [7]:
itson = pd.read_excel('잇츠온.xlsx')
itson['year'] = itson['date'].map(year_extract)
itson = itson.iloc[:, [0,1,7,8,-1]]
itson.columns = ['company', 'product_name', 'score', 'review', 'year']
itson = itson.loc[itson['year'] < 2022, :]
itson = itson.loc[itson['year'] > 2018, :].reset_index(drop = True)
itson.head()

C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\wook\anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,company,product_name,score,review,year
0,fredit,차돌박이 순두부찌개,5.0,맛있어요~,2021
1,fredit,차돌박이 순두부찌개,5.0,순두부가 부드러워 좋네요,2021
2,fredit,차돌박이 순두부찌개,5.0,맛있게 잘 먹었어요,2021
3,fredit,차돌박이 순두부찌개,5.0,좋아요,2021
4,fredit,차돌박이 순두부찌개,5.0,맛나요,2021


In [8]:
data = pd.concat([cookit, fresheasy, mychef, itson]).reset_index(drop = True)
data.head()

,company,product_name,score,review,year
0,cookit,[사전예약] 스파이시 보일링랍스터&쉬림프,3.0,기대가 너무 커서 그랬는지 제 입맛엔 그냥 ...쿡킷 메뉴 대부분 마음에 드는데 이...,2021
1,cookit,[사전예약] 리코타치즈 콜드파스타,5.0,아 이거 진짜 맛있네요!남편과 저는 물론이고 5살 딸도 너무너무 맛있게 잘 먹었어요...,2021
2,cookit,수삼양지수육전골,5.0,추천해요~국물도 따뜻하고 버섯이랑 고기 넉넉해서 한끼 잘 먹었습니다!! 맛있게 잘 ...,2021
3,cookit,평양식 어복쟁반,5.0,맛있게 먹고 재주문해요아이부터 어른까지 온가족 맛있게 먹었어요레시피 보고 삶은달걀인...,2021
4,cookit,감바스알아히요,5.0,감바스 재료를 따로 사지않고 한번에 만들어 먹을 수 있어서 좋아요 !살짝 매콤하게 ...,2021


#### 공란 데이터 제거

In [9]:
data = data.dropna(subset = ['product_name']).reset_index(drop = True)
data = data.dropna(subset = ['review']).reset_index(drop = True)

#### review가 두글자 이상인 경우만 남기기

In [10]:
sample = []
for i in range(len(data['review'])):
    if len(data['review'][i]) > 1:
        sample.append(i)

In [11]:
data = data.iloc[sample, :].reset_index(drop = True)

#### 10개 이상의 리뷰가 있는 제품만 남기기

In [12]:
product_count = pd.DataFrame.from_dict( Counter(data['product_name']), orient='index').reset_index()
product_count = product_count.loc[product_count[0] > 9, :]
product_list = list(product_count['index'])

In [13]:
data = data.loc[data['product_name'].isin(product_list), :].reset_index(drop = True)

In [14]:
len(data['product_name'])

211535

### 3. Tokenize

#### Textmining을 하기위해서는 List형태로 단어들은 Tokenize 해야 합니다.

#### (1) 합성어 전처리

In [15]:
# 동의어 처리
Comb_words = [['에어프라이기'], ['프레시지'], ['배불리'], ['비쥬얼'], ['구매평'], ['밀키트'], ['유린기'], ['질리다'], ['마이셰프', '마이쉐프'], ['프라이팬'], ['쿠킷', '쿳킷'], ['유통기한'], ['알리오올리오'], ['비리다'], ['파스타'], ['푸팟퐁커리'], ['먹어보지'], ['취향저격'], ['우육면'], ['제입'], ['전체적'], ['건고추'], ['강추'], ['초무침'], ['구성품'], ['정기배송'], ['감바스'], ['가성비'], ['엄지척'], ['구매평'], ['푸짐'], ['쉬림프'], ['CJ', 'cj'], ['새콤달콤'], ['플레이팅']]


In [16]:
Transform_words = ['TRSF' * (i+1) for i in range(len(Comb_words))]
Transform_words[0:3]

['TRSF', 'TRSFTRSF', 'TRSFTRSFTRSF']

In [17]:
def Word_transform(Text, word_list, to_word):
    for word in word_list:
        Text = str(Text).replace(word,to_word)
    return Text

In [18]:
for i in range(len(Comb_words)):
    data['review'] = [Word_transform(abstract, Comb_words[i], Transform_words[i]) for abstract in data['review']]    

#### (2) Tokenize

In [19]:
Okt = Okt()

In [46]:
# 명사
#tags = ['Noun', 'Alpha']

# 명사, 형용사
tags = ['Noun']
# 불용어 처리
stopwords = ['입니다','완전','전부','같아요','대한', '있어서', '있는', '약간', '있어요', '살짝', '적당히', '그냥', '있어', '역시', '모두', '있습니다', '다만', '보고', '같은', '있고', '편이', '같습니다', '좀더', 'jpg', '첨부파일', '때문', '일단', '리뷰', '이서', '그런지', '이상', '가장', '바로', '이건', '사서', '충분히', '안나', '없는', '같네요', '없어서', '있네요', '있었어요', '기도', '없고', '있는데', '거의', '무엇', '하니', '있으면', '없네요', '없어요', '전혀', '추합니다', '이런', '같아서', '아니라', '워낙', '같아', '여기', '뭔가', '해도', '있을', '있던', '비주', '위해', '우선', '불리', '있으니', '있지만', '대로', '사실', '같은데', 'jpeg', '없을', '있었습니다', '있게', '있었으면', '기지', '있음',' 같고', '자꾸', '있다니', '없어', '그런', '그것', '짐해', '수도', '미가', '다가', '인지', '있었는데', '듭니', '만해', '없습니다', '이기', '없었어요', '그닥', '그게', '내기', '편입', '런가', '끼리', '기고', '혹시', '그랬어요', '그거', '질도', '서도', '있었지만', '처럼', '이예', '문해', '고요', '이면', '이나', '알도', '그럴', '이구', '한수', '입니당', '그렇고', '거리', '있었네요', '만하', '가요', '있구요', '어요', '있는거', '주시', '수가', '없는데', '나니', '번은', '는걸', '요건', '어떤', '야해요', '있을걸', '단지', '아예', '성하게', '있다면', '있다는', '고해', '드네', '양장', '없었는데', '로만', '있었음', '있다', '리오', '시기', '어도', '비도', '점점', '없지만', '보이', '차라리', '하라', '그래요', '그렇지', '같기도', '거나', '등등', '더더']


def Tokenizer(text):
    morphs = Okt.pos(text)
    if len(morphs) > 0:
        pos = []

        for x in morphs:
            if len(x) > 1:
                word, tag = x
                if(word in stopwords): 
                    continue
                if tag in tags:
                    pos.append(word)
    else:
        pos = ['nan']
    return pos

In [47]:
Token = [Tokenizer(text) for text in data['review']]
Token[0:3]

[['기대', '커서', '제', '입맛', '쿡킷', '메뉴', '대부분', '마음', '테', '맛'],
 ['거',
  '진짜',
  '남편',
  '저',
  '살',
  '딸도',
  '상큼',
  '함',
  '말',
  '식감',
  '푸른',
  '채소',
  '리코',
  '신의',
  '리코',
  '치즈',
  '리코',
  '치즈',
  '통',
  '뭐',
  '젓가락',
  '질',
  '때',
  '계속',
  '크리스마스',
  '외식',
  '마음',
  '걸',
  '방'],
 ['추천', '국물', '버섯', '고기', '끼']]

#### (3) 앞서 바꿔준 합성어를 다시 원상태로 복구합니다

In [48]:
def Word_restore(Text, word1, word2):
    transformed_list = []
    for word in Text:
        if word1 in word:
            word = word2
        transformed_list.append(word)
    return transformed_list

In [49]:
Comb_words = [word_list[0] for word_list in Comb_words]
for i in range(1, len(Comb_words) + 1):
    Token = [Word_restore(token, Transform_words[-i], Comb_words[-i]) for token in Token] 
Token[0:3]

[['기대', '커서', '제', '입맛', '쿡킷', '메뉴', '대부분', '마음', '테', '맛'],
 ['거',
  '진짜',
  '남편',
  '저',
  '살',
  '딸도',
  '상큼',
  '함',
  '말',
  '식감',
  '푸른',
  '채소',
  '리코',
  '신의',
  '리코',
  '치즈',
  '리코',
  '치즈',
  '통',
  '뭐',
  '젓가락',
  '질',
  '때',
  '계속',
  '크리스마스',
  '외식',
  '마음',
  '걸',
  '방'],
 ['추천', '국물', '버섯', '고기', '끼']]

#### (4) 한 글자 단어들도 제거해줄 수 있습니다.

In [27]:
def one_char_remove(Token):
    
    Token = [word for word in Token if (len(word) > 1 or word == '맛')]
    return Token

In [25]:
def one_char_remove(Token):
    Token = [word for word in Token if len(word) > 1]
    return Token

In [50]:
def one_char_remove(Token):
    Token = [word for word in Token if word in list(word_count['Word'])]
    return Token

In [45]:
'맛' in list(word_count['Word'])

False

In [51]:
Token = [one_char_remove(token) for token in Token]
Token[0:20]

[['기대', '커서', '입맛', '쿡킷', '메뉴', '대부분', '마음', '맛'],
 ['진짜',
  '남편',
  '딸도',
  '상큼',
  '식감',
  '채소',
  '신의',
  '치즈',
  '치즈',
  '젓가락',
  '계속',
  '크리스마스',
  '외식',
  '마음'],
 ['추천', '국물', '버섯', '고기'],
 ['재주', '아이', '어른', '가족', '레시피', '달걀'],
 ['재료', '사지', '한번', '매콤'],
 ['아이', '크리스마스', '분위기', '요리', '가족'],
 ['재료', '겨울', '맛', '해산물', '하나', '재미', '매콤', '콩나물', '그릇', '금방', '뚝딱'],
 ['양은', '조절', '추가', '사용', '수비드', '치킨', '추천'],
 ['만들기', '맛', '혼자', '오븐', '진짜', '개발', '한번', '구매'],
 ['고생', '부모님', '생각', '친정', '전화'],
 ['재료', '만들기', '피자', '맛'],
 ['부채살', '스테이크', '기름', '고기'],
 ['쿡킷', '맛', '요리', '수비드', '사태', '갈비찜'],
 ['음식', '메뉴', '아주', '재료'],
 ['피자', '치즈', '또띠아', '남아', '피자', '치즈', '듬뿍'],
 ['고기', '기름', '기름', '고기', '이적', '야채'],
 ['다음', '의사', '나중'],
 ['레시피', '따라서', '가족', '식사'],
 ['고등학생', '아이', '레시피', '요리', '고등어', '조합'],
 ['레시피', '따라서', '가족', '식사']]

In [60]:
flatten_tokens = list(itertools.chain(*Token))
token_count = Counter(flatten_tokens)
Top_token_count = token_count.most_common(2000)
    
word_count = pd.DataFrame(Top_token_count)
word_count.columns = ['Word', 'Count']

In [61]:
word_count

,Word,Count
0,맛,83440
1,고기,29602
2,주문,28082
3,조리,27806
4,요리,26828
...,...,...
1995,앞뒤,48
1996,낼때,48
1997,매뉴얼,48
1998,사전예약,48


In [41]:
flatten_tokens

[]

### 4. Wordcloud

In [ ]:
def visulize(noun_list, filename, white = True):
    if white == True:
        wordcloud = WordCloud(width=2000, height=1000, font_path = 'C:/Windows/Fonts/malgun.ttf', collocations=True, background_color='white')
    else:
        wordcloud = WordCloud(width=2000, height=1000, font_path = 'C:/Windows/Fonts/malgun.ttf', collocations=True)
    wordcloud.generate_from_frequencies(dict(noun_list))
    wordcloud.to_file(filename)

In [ ]:
def wordcloud_Rank(token, N = 30, white = True):
    if not os.path.exists('./Wordcloud'):
        os.makedirs('./Wordcloud')
    filename = 'Wordcloud/keyword_wordcloud.png'
    filename_csv = 'Wordcloud/keyword_count.csv'
    
    flatten_tokens = list(itertools.chain(*token))
    token_count = Counter(flatten_tokens)
    Top_token_count = token_count.most_common(N)
    
    word_count = pd.DataFrame(Top_token_count)
    word_count.columns = ['Word', 'Count']
    word_count.to_csv(filename_csv, index = False, encoding='euc-kr')
    
    visulize(Top_token_count, filename, white)
    return word_count

In [ ]:
word_count = wordcloud_Rank(Token, 50, white = False)
word_count.head(10)

## 5. TF-IDF

In [52]:
data['Precessed_text'] = ''
for ind in range(len(Token)):
    data['Precessed_text'][ind] = ' '.join(Token[ind])
data.head()

<ipython-input-52-f33233f7e71e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Precessed_text'][ind] = ' '.join(Token[ind])


,company,product_name,score,review,year,Precessed_text
0,cookit,[사전예약] 스파이시 보일링랍스터&쉬림프,3.0,기대가 너무 커서 그랬는지 제 입맛엔 그냥 ...쿡킷 메뉴 대부분 마음에 드는데 이...,2021,기대 커서 입맛 쿡킷 메뉴 대부분 마음 맛
1,cookit,[사전예약] 리코타치즈 콜드파스타,5.0,아 이거 진짜 맛있네요!남편과 저는 물론이고 5살 딸도 너무너무 맛있게 잘 먹었어요...,2021,진짜 남편 딸도 상큼 식감 채소 신의 치즈 치즈 젓가락 계속 크리스마스 외식 마음
2,cookit,수삼양지수육전골,5.0,추천해요~국물도 따뜻하고 버섯이랑 고기 넉넉해서 한끼 잘 먹었습니다!! 맛있게 잘 ...,2021,추천 국물 버섯 고기
3,cookit,평양식 어복쟁반,5.0,맛있게 먹고 재주문해요아이부터 어른까지 온가족 맛있게 먹었어요레시피 보고 삶은달걀인...,2021,재주 아이 어른 가족 레시피 달걀
4,cookit,감바스알아히요,5.0,TRSFTRSFTRSFTRSFTRSFTRSFTRSFTRSFTRSFTRSFTRSFTR...,2021,재료 사지 한번 매콤


In [53]:
data.loc[:, ['company', 'Precessed_text']].to_csv('tf-idf_data.csv', index = False, encoding = 'euc-kr')

In [54]:
doc_num = len(data['Precessed_text'])
doc_num

211535

In [55]:
cv = CountVectorizer()
tdm = cv.fit_transform(data['Precessed_text'])
tdm = tdm.toarray()

In [56]:
words = cv.get_feature_names()
word_count = tdm.sum(axis=0)

C:\Users\wook\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [57]:
tdm[tdm > 1] = 1
word_doc = tdm.sum(axis=0)

In [58]:
tf = word_count / doc_num
idf = np.log(doc_num / word_doc)
tf_idf = tf * idf

In [59]:
TF_IDF = pd.DataFrame({'Word': words, 'Score': tf_idf})
TF_IDF = TF_IDF.sort_values(by=['Score'], axis=0, ascending=False)
TF_IDF.head(30)

,Word,Score
114,고기,0.297999
1558,주문,0.285046
1285,요리,0.282026
1530,조리,0.279169
949,소스,0.273179
1452,재료,0.270027
1108,아이,0.254161
586,메뉴,0.243109
1143,양도,0.230313
1506,정말,0.218824


In [62]:
TF_IDF = TF_IDF.reset_index(drop = True)
TF_IDF.to_csv('TF_IDF.csv', index = False, encoding='euc-kr')

### 5. Topic Modeling

#### (1) LDA에 필요한 환경설정

In [ ]:
id2word=corpora.Dictionary(Token)
id2word.filter_extremes(no_below = 3) #3회 이하로 등장한 단어는 삭제
texts = Token
corpus=[id2word.doc2bow(text) for text in texts]

#### (2) LDA는 토픽 수를 정해주어야 하며, 이 때, Coherence 값이 높은 K로 정하는게 일반적입니다.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = gensim.models.coherencemodel.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    
    x = range(start, limit, step) 
    plt.plot(x, coherence_values) 
    plt.xlabel("Num Topics") 
    plt.ylabel("Coherence score") 
    plt.legend(("coherence_values"), loc='best') 
    plt.show()
    
    return model_list, coherence_values

#### 그래프에서 y축이 가장 높은 K가 최적의 토픽 개수

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=3, limit=11, step=1)

In [ ]:
coherence_values

#### (3) 최적의 토픽 개수를 찾아, 이에 맞게 토픽모델링

In [ ]:
K = 6
num_words = 30
# model = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model_list[2].print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model1 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model1.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model2 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model2.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model3 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model3.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model4 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model4.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model5 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model5.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model6 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model6.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model7 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model7.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model8 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model8.print_topics(num_words=num_words) 
topics

In [ ]:
K = 6
num_words = 30
model9 = LdaModel(corpus=corpus, num_topics=K, id2word=id2word)
topics = model9.print_topics(num_words=num_words) 
topics

In [ ]:
prob = np.zeros((len(model[corpus]), 6))
for doc in range(len(model[corpus])):
    for topic, proba in model[corpus][doc]:
        prob[doc, topic] = proba
        
prob = pd.DataFrame(prob)
prob.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6']
prob.to_csv('topic_porb.csv', index = False)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model_list[2], corpus, id2word)
vis

### 6. Doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(Token)]

In [ ]:
model = Doc2Vec(documents, vector_size=128, window=5, epochs=100, min_count=5, workers=4)

In [ ]:
def Similar_words(Doc2vec_model, Keyword, N):
    Similar_keywords = Doc2vec_model.wv.most_similar(Keyword, topn = N)
    Keywords = [word for (word, similarity) in Similar_keywords]
    Rank = list(range(1, N + 1))
    Result = pd.DataFrame({'Rank' : Rank, 'Keyword' : Keywords})
    return Result

In [ ]:
keyword = '배송'
Similar_words_list = Similar_words(model, keyword, 100)
Similar_words_list.to_csv(keyword + '관련단어.csv', index = False, encoding = 'euc-kr')

In [ ]:
keyword = '가격'
Similar_words_list = Similar_words(model, keyword, 100)
Similar_words_list.to_csv(keyword + '관련단어.csv', index = False, encoding = 'euc-kr')

In [ ]:
keyword = '재료'
Similar_words_list = Similar_words(model, keyword, 100)
Similar_words_list.to_csv(keyword + '관련단어.csv', index = False, encoding = 'euc-kr')

In [ ]:
keyword = '시간'
Similar_words_list = Similar_words(model, keyword, 100)
Similar_words_list.to_csv(keyword + '관련단어.csv', index = False, encoding = 'euc-kr')

In [ ]:
keyword = '맛'
Similar_words_list = Similar_words(model, keyword, 100)
Similar_words_list.to_csv(keyword + '관련단어.csv', index = False, encoding = 'euc-kr')

#### Doc2vec 결과를 Dataframe으로 저장해봅시다.

#### - Word 저장

In [ ]:
my_words = []
for idx, key in enumerate(model.wv.vocab):
    my_words = my_words + [key]

In [ ]:
Wordvec_result = pd.DataFrame({'Keyword': my_words})
Wordvec_result = pd.concat([Wordvec_result,pd.DataFrame(model[my_words])], axis=1)
Wordvec_result.head()

In [ ]:
Wordvec_result.to_csv('doc2vec_result_word.csv', index = False, encoding='euc-kr')

#### - Document 저장

In [ ]:
doc_vec_array = model.docvecs[0]
for i in range(1, len(documents)):
    doc_vec_array = np.vstack([doc_vec_array, model.docvecs[i]])
doc_vec_array.shape

In [ ]:
doc_vec_df = pd.DataFrame(doc_vec_array)
doc_vec_df.to_csv('doc2vec_result_doc.csv', index = False, encoding='euc-kr')
doc_vec_df.head()

#### - Randomforest 적합

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(doc_vec_array, data['score'])

In [ ]:
score = model.predict(Wordvec_result.iloc[:, 1:])

In [ ]:
sentiment_score = pd.DataFrame({'Keyword': Wordvec_result['Keyword'], 'Score' : score})
sentiment_score.head()

In [ ]:
sentiment_score.to_csv('sentiment_score.csv', index = False, encoding = 'euc-kr')

In [ ]:
sentiment_score = pd.read_csv('sentiment_score.csv', encoding = 'euc-kr')

In [ ]:
sentiment_score['Score'] = 1 +  (max(sentiment_score['Score']) - sentiment_score['Score']) / (max(sentiment_score['Score']) - min(sentiment_score['Score'])) * 4
import math
sentiment_score['Score'] = sentiment_score['Score'].map(round)

In [ ]:
Counter(sentiment_score['Score'])

## 7. 토픽 감성 점수 계산

In [19]:
topic_porb = pd.read_csv('topic_porb_mk.csv', encoding = 'euc-kr')
topic_porb['Score'] = data['score']

In [20]:
topic_porb.head()

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Score
0,0.016954,0.175372,0.016795,0.393785,0.269118,0.127975,3.0
1,0.431590,0.328817,0.097755,0.088636,0.000000,0.046433,5.0
2,0.020890,0.021022,0.707705,0.021015,0.208400,0.020968,5.0
3,0.018545,0.626032,0.018925,0.018650,0.018624,0.299224,5.0
4,0.024013,0.562663,0.024059,0.024084,0.023932,0.341248,5.0


In [35]:
from statsmodels.formula.api import ols
model = ols("Score ~ Topic1 + Topic2 + Topic3 + Topic4 + Topic5 + Topic6", data = topic_porb).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Score   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     2265.
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        00:29:10   Log-Likelihood:            -2.0523e+05
No. Observations:              211535   AIC:                         4.105e+05
Df Residuals:                  211528   BIC:                         4.106e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.3455      0.263     12.744      0.000       2.831       3.860
Topic1         1.0739      0.263      4.081      0.000       0.558       1.590
Topic2         1.5554      0.263      5.917      0.000       1.040       2.071
Topic3         1.6377      0.263      6.232      0.000       1.123       2.153
Topic4         1.5482      0.263      5.890      0.000       1.033       2.063
Topic5         0.8582      0.263      3.260      0.001       0.342       1.374
Topic6         1.5360      0.263      5.839      0.000       1.020       2.052
==============================================================================
Omnibus:                   123499.177   Durbin-Watson:                   1.756
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1075667.430
Skew:                          -2.758   Prob(JB):                         0.00
Kurtosis:                      12.571   Cond. No.                         542.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 1 ~ 5 점으로 Scaling

In [36]:
1 + (model.params[1:] - min(model.params[1:])) / (max(model.params[1:]) - min(model.params[1:])) * 4

Topic1    2.107237
Topic2    4.577747
Topic3    5.000000
Topic4    4.541009
Topic5    1.000000
Topic6    4.478112
dtype: float64

## 8. 토빗 모델

In [37]:
import math
import warnings

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats
from scipy.special import log_ndtr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


def split_left_right_censored(x, y, cens):
    counts = cens.value_counts()
    if -1 not in counts and 1 not in counts:
        warnings.warn("No censored observations; use regression methods for uncensored data")
    xs = []
    ys = []

    for value in [-1, 0, 1]:
        if value in counts:
            split = cens == value
            y_split = np.squeeze(y[split].values)
            x_split = x[split].values

        else:
            y_split, x_split = None, None
        xs.append(x_split)
        ys.append(y_split)
    return xs, ys


def tobit_neg_log_likelihood(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1])
    s = params[-1]

    to_cat = []

    cens = False
    if y_left is not None:
        cens = True
        left = (y_left - np.dot(x_left, b))
        to_cat.append(left)
    if y_right is not None:
        cens = True
        right = (np.dot(x_right, b) - y_right)
        to_cat.append(right)
    if cens:
        concat_stats = np.concatenate(to_cat, axis=0) / s
        log_cum_norm = scipy.stats.norm.logcdf(concat_stats)  # log_ndtr(concat_stats)
        cens_sum = log_cum_norm.sum()
    else:
        cens_sum = 0

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        mid = scipy.stats.norm.logpdf(mid_stats) - math.log(max(np.finfo('float').resolution, s))
        mid_sum = mid.sum()
    else:
        mid_sum = 0

    loglik = cens_sum + mid_sum

    return - loglik


def tobit_neg_log_likelihood_der(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1]) # in censReg, not using chain rule as below; they optimize in terms of log(s)
    s = params[-1]

    beta_jac = np.zeros(len(b))
    sigma_jac = 0

    if y_left is not None:
        left_stats = (y_left - np.dot(x_left, b)) / s
        l_pdf = scipy.stats.norm.logpdf(left_stats)
        l_cdf = log_ndtr(left_stats)
        left_frac = np.exp(l_pdf - l_cdf)
        beta_left = np.dot(left_frac, x_left / s)
        beta_jac -= beta_left

        left_sigma = np.dot(left_frac, left_stats)
        sigma_jac -= left_sigma

    if y_right is not None:
        right_stats = (np.dot(x_right, b) - y_right) / s
        r_pdf = scipy.stats.norm.logpdf(right_stats)
        r_cdf = log_ndtr(right_stats)
        right_frac = np.exp(r_pdf - r_cdf)
        beta_right = np.dot(right_frac, x_right / s)
        beta_jac += beta_right

        right_sigma = np.dot(right_frac, right_stats)
        sigma_jac -= right_sigma

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        beta_mid = np.dot(mid_stats, x_mid / s)
        beta_jac += beta_mid

        mid_sigma = (np.square(mid_stats) - 1).sum()
        sigma_jac += mid_sigma

    combo_jac = np.append(beta_jac, sigma_jac / s)  # by chain rule, since the expression above is dloglik/dlogsigma

    return -combo_jac


class TobitModel:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.ols_coef_ = None
        self.ols_intercept = None
        self.coef_ = None
        self.intercept_ = None
        self.sigma_ = None

    def fit(self, x, y, cens, verbose=False):
        """
        Fit a maximum-likelihood Tobit regression
        :param x: Pandas DataFrame (n_samples, n_features): Data
        :param y: Pandas Series (n_samples,): Target
        :param cens: Pandas Series (n_samples,): -1 indicates left-censored samples, 0 for uncensored, 1 for right-censored
        :param verbose: boolean, show info from minimization
        :return:
        """
        x_copy = x.copy()
        if self.fit_intercept:
            x_copy.insert(0, 'intercept', 1.0)
        else:
            x_copy.scale(with_mean=True, with_std=False, copy=False)
        init_reg = LinearRegression(fit_intercept=False).fit(x_copy, y)
        b0 = init_reg.coef_
        y_pred = init_reg.predict(x_copy)
        resid = y - y_pred
        resid_var = np.var(resid)
        s0 = np.sqrt(resid_var)
        params0 = np.append(b0, s0)
        xs, ys = split_left_right_censored(x_copy, y, cens)

        result = minimize(lambda params: tobit_neg_log_likelihood(xs, ys, params), params0, method='BFGS',
                          jac=lambda params: tobit_neg_log_likelihood_der(xs, ys, params), options={'disp': verbose})
        if verbose:
            print(result)
        self.ols_coef_ = b0[1:]
        self.ols_intercept = b0[0]
        if self.fit_intercept:
            self.intercept_ = result.x[1]
            self.coef_ = result.x[1:-1]
        else:
            self.coef_ = result.x[:-1]
            self.intercept_ = 0
        self.sigma_ = result.x[-1]
        return self

    def predict(self, x):
        return self.intercept_ + np.dot(x, self.coef_)

    def score(self, x, y, scoring_function=mean_absolute_error):
        y_pred = np.dot(x, self.coef_)
        return scoring_function(y, y_pred)

In [42]:
tobit = TobitModel().fit(topic_porb.iloc[:, :-1], topic_porb['Score'], topic_porb['Score'])
result = pd.DataFrame({"Topic" : ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6'], 'Coef': tobit.coef_})
result

,Topic,Coef
0,Topic1,1.077015
1,Topic2,1.556538
2,Topic3,1.638573
3,Topic4,1.549865
4,Topic5,0.867329
5,Topic6,1.537196


#### 1 ~ 5 점으로 Scaling

In [43]:
coef = 1 + (tobit.coef_ - min(tobit.coef_)) / (max(tobit.coef_) - min(tobit.coef_)) * 4
result = pd.DataFrame({"Topic" : ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6'], 'Coef': coef})
result

,Topic,Coef
0,Topic1,2.087523
1,Topic2,4.574531
2,Topic3,5.000000
3,Topic4,4.539923
4,Topic5,1.000000
5,Topic6,4.474214
